In [1]:
import sys
sys.path.append('..')

In [2]:
import os
os.getcwd()

'/mnt/3B1D7BDC2D2641B0/Documents/GIT/SQUAD_NLP/pythonbooks'

In [3]:
os.chdir('..')
os.getcwd()

'/mnt/3B1D7BDC2D2641B0/Documents/GIT/SQUAD_NLP'

In [4]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


# Хочется вытащить все в питонбук, потому что из консоли работать неудобно

# Обучение

In [5]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as sched
import torch.utils.data as data
import util

from args import get_train_args
from collections import OrderedDict
from json import dumps
from models import BiDAF, BiDAF_CBE
from tensorboardX import SummaryWriter
from tqdm import tqdm
from ujson import load as json_load
from util import collate_fn, SQuAD

In [6]:
train_args = type('', (), {})() # empty "structure" where you can add fields

train_args.train_record_file = './data/train.npz'
train_args.dev_record_file = './data/dev.npz'
train_args.test_record_file = './data/test.npz'
train_args.word_emb_file = './data/word_emb.json'
train_args.char_emb_file = './data/char_emb.json'
train_args.train_eval_file = './data/train_eval.json'
train_args.dev_eval_file = './data/dev_eval.json'
train_args.test_eval_file = './data/test_eval.json'

train_args.name = 'train'       # ???
train_args.max_ans_len = 15     # Maximum length of a predicted answer.
train_args.num_workers = 6      # Number of sub-processes to use per data loader.
train_args.save_dir = './save/' # Base directory for saving information.
train_args.batch_size = 80     # Batch size per GPU. Scales automatically when \
                                #    multiple GPUs are available.
train_args.use_squad_v2 = True  # Whether to use SQuAD 2.0 (unanswerable) questions.
train_args.hidden_size = 100    # Number of features in encoder hidden layers
train_args.num_visuals = 10     # Number of examples to visualize in TensorBoard.
train_args.load_path = None     # Path to load as a model checkpoint.

train_args.eval_steps = 25000   # Number of steps between successive evaluations
train_args.lr = 1.0             # learning rate
train_args.l2_wd = 0.0          # L2 weight decay.
train_args.num_epochs = 30      # Number of epochs for which to train. Negative means forever.
train_args.drop_prob = 0.2      # Probability of zeroing an activation in dropout layers.
train_args.metric_name = 'F1'   # choices=('NLL', 'EM', 'F1'), name of dev metric to determine best checkpoint.
train_args.max_checkpoints = 5  # Maximum number of checkpoints to keep on disk.
train_args.max_grad_norm = 5.0  # Maximum gradient norm for gradient clipping.
train_args.seed = 224           # Random seed
train_args.ema_decay = 0.999    # Decay rate for exponential moving average of parameters.

if train_args.metric_name == 'NLL':
    # Best checkpoint is the one that minimizes negative log-likelihood
    train_args.maximize_metric = False
elif train_args.metric_name in ('EM', 'F1'):
    # Best checkpoint is the one that maximizes EM or F1
    train_args.maximize_metric = True

In [7]:
# from train.py

lambda3 = lambda step: (0.01 + 0.0021 * (step+1)) if step < 1000 else 2.0 if step < 1500 else 0.99988 ** (step - 1500)

In [8]:
for i in range(45500):
    print(i, ' = ', lambda3(i))

0  =  0.0121
1  =  0.0142
2  =  0.016300000000000002
3  =  0.0184
4  =  0.020499999999999997
5  =  0.022600000000000002
6  =  0.0247
7  =  0.026799999999999997
8  =  0.028900000000000002
9  =  0.031
10  =  0.0331
11  =  0.0352
12  =  0.0373
13  =  0.0394
14  =  0.0415
15  =  0.0436
16  =  0.0457
17  =  0.0478
18  =  0.0499
19  =  0.052
20  =  0.0541
21  =  0.0562
22  =  0.0583
23  =  0.0604
24  =  0.0625
25  =  0.06459999999999999
26  =  0.0667
27  =  0.0688
28  =  0.07089999999999999
29  =  0.073
30  =  0.07509999999999999
31  =  0.07719999999999999
32  =  0.0793
33  =  0.08139999999999999
34  =  0.08349999999999999
35  =  0.0856
36  =  0.08769999999999999
37  =  0.08979999999999999
38  =  0.0919
39  =  0.09399999999999999
40  =  0.09609999999999999
41  =  0.0982
42  =  0.10029999999999999
43  =  0.10239999999999999
44  =  0.1045
45  =  0.10659999999999999
46  =  0.10869999999999999
47  =  0.1108
48  =  0.11289999999999999
49  =  0.11499999999999999
50  =  0.11709999999999998
51  =  0

1314  =  2.0
1315  =  2.0
1316  =  2.0
1317  =  2.0
1318  =  2.0
1319  =  2.0
1320  =  2.0
1321  =  2.0
1322  =  2.0
1323  =  2.0
1324  =  2.0
1325  =  2.0
1326  =  2.0
1327  =  2.0
1328  =  2.0
1329  =  2.0
1330  =  2.0
1331  =  2.0
1332  =  2.0
1333  =  2.0
1334  =  2.0
1335  =  2.0
1336  =  2.0
1337  =  2.0
1338  =  2.0
1339  =  2.0
1340  =  2.0
1341  =  2.0
1342  =  2.0
1343  =  2.0
1344  =  2.0
1345  =  2.0
1346  =  2.0
1347  =  2.0
1348  =  2.0
1349  =  2.0
1350  =  2.0
1351  =  2.0
1352  =  2.0
1353  =  2.0
1354  =  2.0
1355  =  2.0
1356  =  2.0
1357  =  2.0
1358  =  2.0
1359  =  2.0
1360  =  2.0
1361  =  2.0
1362  =  2.0
1363  =  2.0
1364  =  2.0
1365  =  2.0
1366  =  2.0
1367  =  2.0
1368  =  2.0
1369  =  2.0
1370  =  2.0
1371  =  2.0
1372  =  2.0
1373  =  2.0
1374  =  2.0
1375  =  2.0
1376  =  2.0
1377  =  2.0
1378  =  2.0
1379  =  2.0
1380  =  2.0
1381  =  2.0
1382  =  2.0
1383  =  2.0
1384  =  2.0
1385  =  2.0
1386  =  2.0
1387  =  2.0
1388  =  2.0
1389  =  2.0
1390  =  2.0

2814  =  0.8541149763247681
2815  =  0.8540124825276091
2816  =  0.8539100010297058
2817  =  0.8538075318295822
2818  =  0.8537050749257626
2819  =  0.8536026303167715
2820  =  0.8535001980011335
2821  =  0.8533977779773734
2822  =  0.8532953702440161
2823  =  0.8531929747995868
2824  =  0.8530905916426108
2825  =  0.8529882207716137
2826  =  0.8528858621851212
2827  =  0.8527835158816589
2828  =  0.8526811818597531
2829  =  0.8525788601179299
2830  =  0.8524765506547157
2831  =  0.8523742534686372
2832  =  0.852271968558221
2833  =  0.852169695921994
2834  =  0.8520674355584833
2835  =  0.8519651874662163
2836  =  0.8518629516437203
2837  =  0.8517607280895231
2838  =  0.8516585168021523
2839  =  0.8515563177801361
2840  =  0.8514541310220024
2841  =  0.8513519565262798
2842  =  0.8512497942914966
2843  =  0.8511476443161816
2844  =  0.8510455065988637
2845  =  0.8509433811380718
2846  =  0.8508412679323353
2847  =  0.8507391669801834
2848  =  0.8506370782801457
2849  =  0.85053500183

4314  =  0.7134090913680725
4315  =  0.7133234822771083
4316  =  0.713237883459235
4317  =  0.71315229491322
4318  =  0.7130667166378304
4319  =  0.7129811486318338
4320  =  0.712895590893998
4321  =  0.7128100434230907
4322  =  0.7127245062178799
4323  =  0.7126389792771337
4324  =  0.7125534625996205
4325  =  0.7124679561841085
4326  =  0.7123824600293664
4327  =  0.7122969741341629
4328  =  0.7122114984972668
4329  =  0.7121260331174472
4330  =  0.712040577993473
4331  =  0.7119551331241138
4332  =  0.7118696985081389
4333  =  0.7117842741443179
4334  =  0.7116988600314206
4335  =  0.7116134561682168
4336  =  0.7115280625534767
4337  =  0.7114426791859702
4338  =  0.7113573060644679
4339  =  0.7112719431877401
4340  =  0.7111865905545576
4341  =  0.711101248163691
4342  =  0.7110159160139113
4343  =  0.7109305941039897
4344  =  0.7108452824326972
4345  =  0.7107599809988053
4346  =  0.7106746898010854
4347  =  0.7105894088383093
4348  =  0.7105041381092487
4349  =  0.710418877612675

5814  =  0.5958829264844725
5815  =  0.5958114205332944
5816  =  0.5957399231628304
5817  =  0.5956684343720509
5818  =  0.5955969541599262
5819  =  0.595525482525427
5820  =  0.595454019467524
5821  =  0.5953825649851878
5822  =  0.5953111190773896
5823  =  0.5952396817431004
5824  =  0.5951682529812912
5825  =  0.5950968327909334
5826  =  0.5950254211709984
5827  =  0.5949540181204579
5828  =  0.5948826236382835
5829  =  0.5948112377234469
5830  =  0.5947398603749201
5831  =  0.5946684915916751
5832  =  0.5945971313726841
5833  =  0.5945257797169193
5834  =  0.5944544366233533
5835  =  0.5943831020909585
5836  =  0.5943117761187076
5837  =  0.5942404587055733
5838  =  0.5941691498505286
5839  =  0.5940978495525466
5840  =  0.5940265578106003
5841  =  0.593955274623663
5842  =  0.5938839999907082
5843  =  0.5938127339107092
5844  =  0.59374147638264
5845  =  0.593670227405474
5846  =  0.5935989869781854
5847  =  0.593527755099748
5848  =  0.593456531769136
5849  =  0.5933853169853237


...

37810  =  0.012810867396473096
37811  =  0.012809330092385518
37812  =  0.012807792972774432
37813  =  0.012806256037617699
37814  =  0.012804719286893184
37815  =  0.012803182720578758
37816  =  0.012801646338652288
37817  =  0.012800110141091649
37818  =  0.012798574127874718
37819  =  0.012797038298979373
37820  =  0.012795502654383496
37821  =  0.012793967194064969
37822  =  0.012792431918001682
37823  =  0.012790896826171521
37824  =  0.01278936191855238
37825  =  0.012787827195122154
37826  =  0.012786292655858738
37827  =  0.012784758300740036
37828  =  0.012783224129743948
37829  =  0.012781690142848377
37830  =  0.012780156340031235
37831  =  0.012778622721270432
37832  =  0.012777089286543879
37833  =  0.012775556035829494
37834  =  0.012774022969105195
37835  =  0.012772490086348901
37836  =  0.012770957387538539
37837  =  0.012769424872652035
37838  =  0.012767892541667316
37839  =  0.012766360394562316
37840  =  0.012764828431314969
37841  =  0.01276329665190321
37842  =  

39477  =  0.010488105680176102
39478  =  0.01048684710749448
39479  =  0.010485588685841582
39480  =  0.010484330415199281
39481  =  0.010483072295549458
39482  =  0.01048181432687399
39483  =  0.010480556509154765
39484  =  0.010479298842373667
39485  =  0.010478041326512583
39486  =  0.010476783961553401
39487  =  0.010475526747478014
39488  =  0.010474269684268317
39489  =  0.010473012771906204
39490  =  0.010471756010373576
39491  =  0.010470499399652331
39492  =  0.010469242939724373
39493  =  0.010467986630571606
39494  =  0.010466730472175936
39495  =  0.010465474464519276
39496  =  0.010464218607583533
39497  =  0.010462962901350624
39498  =  0.01046170734580246
39499  =  0.010460451940920965
39500  =  0.010459196686688054
39501  =  0.010457941583085651
39502  =  0.01045668663009568
39503  =  0.01045543182770007
39504  =  0.010454177175880746
39505  =  0.01045292267461964
39506  =  0.010451668323898685
39507  =  0.010450414123699818
39508  =  0.010449160074004973
39509  =  0.01

41143  =  0.00858751862628099
41144  =  0.008586488124045836
41145  =  0.00858545774547095
41146  =  0.008584427490541495
41147  =  0.00858339735924263
41148  =  0.00858236735155952
41149  =  0.008581337467477333
41150  =  0.008580307706981235
41151  =  0.008579278070056397
41152  =  0.008578248556687991
41153  =  0.008577219166861189
41154  =  0.008576189900561165
41155  =  0.008575160757773098
41156  =  0.008574131738482165
41157  =  0.008573102842673546
41158  =  0.008572074070332426
41159  =  0.008571045421443985
41160  =  0.008570016895993413
41161  =  0.008568988493965892
41162  =  0.008567960215346617
41163  =  0.008566932060120776
41164  =  0.008565904028273561
41165  =  0.008564876119790168
41166  =  0.008563848334655793
41167  =  0.008562820672855635
41168  =  0.008561793134374891
41169  =  0.008560765719198766
41170  =  0.008559738427312462
41171  =  0.008558711258701186
41172  =  0.00855768421335014
41173  =  0.008556657291244538
41174  =  0.00855563049236959
41175  =  0.00

42810  =  0.007030499972836472
42811  =  0.007029656312839731
42812  =  0.0070288127540821905
42813  =  0.007027969296551701
42814  =  0.007027125940236114
42815  =  0.007026282685123286
42816  =  0.0070254395312010715
42817  =  0.007024596478457327
42818  =  0.007023753526879912
42819  =  0.007022910676456686
42820  =  0.007022067927175512
42821  =  0.007021225279024251
42822  =  0.007020382731990767
42823  =  0.007019540286062928
42824  =  0.007018697941228601
42825  =  0.0070178556974756535
42826  =  0.007017013554791956
42827  =  0.007016171513165381
42828  =  0.007015329572583801
42829  =  0.007014487733035091
42830  =  0.007013645994507127
42831  =  0.007012804356987786
42832  =  0.007011962820464947
42833  =  0.007011121384926491
42834  =  0.0070102800503603
42835  =  0.007009438816754257
42836  =  0.007008597684096246
42837  =  0.007007756652374154
42838  =  0.00700691572157587
42839  =  0.007006074891689281
42840  =  0.007005234162702278
42841  =  0.007004393534602754
42842  =

44476  =  0.005756477986574546
44477  =  0.0057557872092161575
44478  =  0.005755096514751051
44479  =  0.005754405903169281
44480  =  0.005753715374460901
44481  =  0.005753024928615966
44482  =  0.005752334565624532
44483  =  0.005751644285476656
44484  =  0.005750954088162399
44485  =  0.00575026397367182
44486  =  0.005749573941994979
44487  =  0.0057488839931219395
44488  =  0.005748194127042765
44489  =  0.00574750434374752
44490  =  0.00574681464322627
44491  =  0.005746125025469083
44492  =  0.005745435490466026
44493  =  0.00574474603820717
44494  =  0.005744056668682586
44495  =  0.0057433673818823435
44496  =  0.005742678177796518
44497  =  0.005741989056415182
44498  =  0.005741300017728412
44499  =  0.005740611061726285
44500  =  0.005739922188398878
44501  =  0.00573923339773627
44502  =  0.005738544689728542
44503  =  0.0057378560643657735
44504  =  0.00573716752163805
44505  =  0.005736479061535453
44506  =  0.005735790684048069
44507  =  0.005735102389165983
44508  =  

In [9]:
def main(args):
    # Set up logging and devices
    args.save_dir = util.get_save_dir(args.save_dir, args.name, training=True)
    log = util.get_logger(args.save_dir, args.name)
    tbx = SummaryWriter(args.save_dir)
    device, args.gpu_ids = util.get_available_devices()
    log.info('Args: {}'.format(dumps(vars(args), indent=4, sort_keys=True)))
    args.batch_size *= max(1, len(args.gpu_ids))

    # Set random seed
    log.info('Using random seed {}...'.format(args.seed))
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)

    # Get embeddings
    log.info('Loading embeddings...')
    word_vectors = util.torch_from_json(args.word_emb_file)

    
    
    # Get data loader
    log.info('Building dataset...')
    train_dataset = SQuAD(args.train_record_file, args.use_squad_v2)
    train_loader = data.DataLoader(train_dataset,
                                   batch_size=args.batch_size,
                                   shuffle=True,
                                   num_workers=args.num_workers,
                                   collate_fn=collate_fn)
    dev_dataset = SQuAD(args.dev_record_file, args.use_squad_v2)
    dev_loader = data.DataLoader(dev_dataset,
                                 batch_size=args.batch_size,
                                 shuffle=False,
                                 num_workers=args.num_workers,
                                 collate_fn=collate_fn)
    
    
    # Get model
    log.info('Building model...')
    model = BiDAF_CBE(word_vectors=word_vectors,
                      hidden_size=args.hidden_size,
                      drop_prob=args.drop_prob,
                      char_dict_size=1400,#train_dataset.get_char_dict_size(),
                      char_emb_size=args.hidden_size,
                      conv_kernel_size=3,
                      conv_depth1=50,
                      conv_output_hidden_size=args.hidden_size)
#     model.summary()
    model = nn.DataParallel(model, args.gpu_ids)
    if args.load_path:
        log.info('Loading checkpoint from {}...'.format(args.load_path))
        model, step = util.load_model(model, args.load_path, args.gpu_ids)
    else:
        step = 0
    model = model.to(device)
    model.train()
   
    ema = util.EMA(model, args.ema_decay)

    # Get saver
    saver = util.CheckpointSaver(args.save_dir,
                                 max_checkpoints=args.max_checkpoints,
                                 metric_name=args.metric_name,
                                 maximize_metric=args.maximize_metric,
                                 log=log)

    # Get optimizer and scheduler
    optimizer = optim.Adadelta(model.parameters(), args.lr,
                               weight_decay=args.l2_wd)
    scheduler = sched.LambdaLR(optimizer, lambda3)  # Constant LR

    

    # Train
    log.info('Training...')
    steps_till_eval = args.eval_steps
    steps_till_print_lr = 5000
    epoch = step // len(train_dataset)
    while epoch != args.num_epochs:
        epoch += 1
        log.info('Starting epoch {}...'.format(epoch))
        with torch.enable_grad(), \
                tqdm(total=len(train_loader.dataset)) as progress_bar:
            for cw_idxs, cc_idxs, qw_idxs, qc_idxs, y1, y2, ids in train_loader:
                # Setup for forward
                cw_idxs = cw_idxs.to(device)
                cc_idxs = cc_idxs.to(device)
                qw_idxs = qw_idxs.to(device)
                qc_idxs = qc_idxs.to(device)
                batch_size = cw_idxs.size(0)
                optimizer.zero_grad()

                # Forward
                log_p1, log_p2 = model(cw_idxs, cc_idxs, qw_idxs, qc_idxs)
                y1, y2 = y1.to(device), y2.to(device)
                loss = F.nll_loss(log_p1, y1) + F.nll_loss(log_p2, y2)
                loss_val = loss.item()

                # Backward
                loss.backward()
                nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()
                
                
                ema(model, step // batch_size)

                # Log info
                step += batch_size
                progress_bar.update(batch_size)
                progress_bar.set_postfix(epoch=epoch,
                                         NLL=loss_val)
                tbx.add_scalar('train/NLL', loss_val, step)
                tbx.add_scalar('train/LR',
                               optimizer.param_groups[0]['lr'],
                               step)

                steps_till_eval -= batch_size
                
                steps_till_print_lr -= batch_size
                if steps_till_print_lr <= 0:
                    print('Learning rate: ', end='')
                    for param_group in optimizer.param_groups:
                        print(param_group['lr'])
                    steps_till_print_lr = 5000
                if steps_till_eval <= 0:
                    steps_till_eval = args.eval_steps

                    # Evaluate and save checkpoint
                    log.info('Evaluating at step {}...'.format(step))
                    #ema.assign(model)
                    results, pred_dict = evaluate(model, dev_loader, device,
                                                  args.dev_eval_file,
                                                  args.max_ans_len,
                                                  args.use_squad_v2)
                    saver.save(step, model, results[args.metric_name], device)
                    #ema.resume(model)

                    # Log to console
                    results_str = ', '.join('{}: {:05.2f}'.format(k, v)
                                            for k, v in results.items())
                    log.info('Dev {}'.format(results_str))

                    # Log to TensorBoard
                    log.info('Visualizing in TensorBoard...')
                    for k, v in results.items():
                        tbx.add_scalar('dev/{}'.format(k), v, step)
                    util.visualize(tbx,
                                   pred_dict=pred_dict,
                                   eval_path=args.dev_eval_file,
                                   step=step,
                                   split='dev',
                                   num_visuals=args.num_visuals)

In [10]:
def evaluate(model, data_loader, device, eval_file, max_len, use_squad_v2):
    nll_meter = util.AverageMeter()

    model.eval()
    pred_dict = {}
    with open(eval_file, 'r') as fh:
        gold_dict = json_load(fh)
    with torch.no_grad(), \
            tqdm(total=len(data_loader.dataset)) as progress_bar:
        for cw_idxs, cc_idxs, qw_idxs, qc_idxs, y1, y2, ids in data_loader:
            # Setup for forward
            cw_idxs = cw_idxs.to(device)
            cc_idxs = cc_idxs.to(device)
            qw_idxs = qw_idxs.to(device)
            qc_idxs = qc_idxs.to(device)
            batch_size = cw_idxs.size(0)

            # Forward
            log_p1, log_p2 = model(cw_idxs, cc_idxs, qw_idxs, qc_idxs)
            y1, y2 = y1.to(device), y2.to(device)
            loss = F.nll_loss(log_p1, y1) + F.nll_loss(log_p2, y2)
            nll_meter.update(loss.item(), batch_size)

            # Get F1 and EM scores
            p1, p2 = log_p1.exp(), log_p2.exp()
            starts, ends = util.discretize(p1, p2, max_len, use_squad_v2)

            # Log info
            progress_bar.update(batch_size)
            progress_bar.set_postfix(NLL=nll_meter.avg)

            preds, _ = util.convert_tokens(gold_dict,
                                           ids.tolist(),
                                           starts.tolist(),
                                           ends.tolist(),
                                           use_squad_v2)
            pred_dict.update(preds)

    model.train()

    results = util.eval_dicts(gold_dict, pred_dict, use_squad_v2)
    results_list = [('NLL', nll_meter.avg),
                    ('F1', results['F1']),
                    ('EM', results['EM'])]
    if use_squad_v2:
        results_list.append(('AvNA', results['AvNA']))
    results = OrderedDict(results_list)

    return results, pred_dict

In [11]:
main(train_args)

[05.06.19 03:58:51] Args: {
    "batch_size": 80,
    "char_emb_file": "./data/char_emb.json",
    "dev_eval_file": "./data/dev_eval.json",
    "dev_record_file": "./data/dev.npz",
    "drop_prob": 0.2,
    "ema_decay": 0.999,
    "eval_steps": 25000,
    "gpu_ids": [
        0
    ],
    "hidden_size": 100,
    "l2_wd": 0.0,
    "load_path": null,
    "lr": 1.0,
    "max_ans_len": 15,
    "max_checkpoints": 5,
    "max_grad_norm": 5.0,
    "maximize_metric": true,
    "metric_name": "F1",
    "name": "train",
    "num_epochs": 30,
    "num_visuals": 10,
    "num_workers": 6,
    "save_dir": "./save/train/train-79",
    "seed": 224,
    "test_eval_file": "./data/test_eval.json",
    "test_record_file": "./data/test.npz",
    "train_eval_file": "./data/train_eval.json",
    "train_record_file": "./data/train.npz",
    "use_squad_v2": true,
    "word_emb_file": "./data/word_emb.json"
}
[05.06.19 03:58:51] Using random seed 224...
[05.06.19 03:58:51] Loading embeddings...
[05.06.19 03:58:

  0%|          | 0/129922 [00:00<?, ?it/s]

[05.06.19 03:59:50] Saver will maximize F1...
[05.06.19 03:59:50] Training...
[05.06.19 03:59:50] Starting epoch 1...


  4%|▍         | 5040/129922 [01:17<31:34, 65.92it/s, NLL=8.03, epoch=1]

Learning rate: 0.1423


...

Learning rate: 0.04227202236426565


 15%|█▌        | 19920/129922 [04:59<28:04, 65.31it/s, NLL=1.61, epoch=18]

[05.06.19 13:32:13] Evaluating at step 2228594...



  0%|          | 0/5951 [00:00<?, ?it/s]


OSError: [Errno 12] Cannot allocate memory

In [9]:
torch.backends.cudnn.version()

7301

# посмотрим, из чего состоят cc_idxs

In [7]:
train_dataset = SQuAD(train_args.train_record_file, train_args.use_squad_v2)
train_loader = data.DataLoader(train_dataset,
                               batch_size=train_args.batch_size,
                               shuffle=True,
                               num_workers=train_args.num_workers,
                               collate_fn=collate_fn)

In [8]:
train_dataset.get_char_dict_size()

1346

In [22]:
jjj = 0
cset1 = set()
cset2 = set()
for cw_idxs, cc_idxs, qw_idxs, qc_idxs, y1, y2, ids in train_loader:
    jjj += 1
#     print(cc_idxs.shape)
#     print(cc_idxs.numpy())
    cset1 |= set(cc_idxs.numpy().ravel())
    cset2 |= set(qc_idxs.numpy().ravel())
#     print(cc_idxs[0, 1, :])
#     if jjj > 100:
#         break

In [23]:
''.join([str(x) + ' ' for x in list(cset1)])

'0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 245 247 248 249 250 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 2

In [24]:
''.join([str(x) + ' ' for x in list(cset2)])

'0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 534 535 25 538 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 523 77 78 79 80 81 594 82 76 83 86 87 88 89 84 91 604 85 90 1110 94 609 528 527 101 102 104 105 106 107 108 109 110 111 112 113 114 115 628 117 119 631 120 122 121 642 131 646 139 140 653 143 144 655 146 661 662 663 1175 1176 163 165 166 542 167 678 172 176 516 189 517 220 230 231 1274 253 259 260 262 263 1288 265 266 1289 1292 1291 270 1290 272 273 276 279 282 290 291 810 303 306 309 310 311 312 825 313 321 322 324 325 326 327 329 330 331 332 333 334 336 340 343 345 348 1372 1374 1375 353 355 356 359 360 361 366 367 373 379 384 587 589 409 590 591 1100 422 423 424 440 441 442 443 478 479 480 481 483 486 1002 498 501 '

# тест

In [22]:
aaaa = np.array([[0, 1], [1, 2]]).ravel().tolist()

In [23]:
aaaa

[0, 1, 1, 2]

In [24]:
set(aaaa)

{0, 1, 2}

In [5]:
util.get_available_devices()

(device(type='cuda', index=0), [0])

# тест2

In [6]:
device, train_args.gpu_ids = util.get_available_devices()

In [7]:
device, train_args.gpu_ids

(device(type='cuda', index=0), [0])

In [10]:
A = torch.rand((19456, 1, 16, 100)).to(device)

In [12]:
B = nn.Conv2d(in_channels = 1, 
                out_channels = 72, 
                kernel_size = (3, 100),
                padding=[1, 0]).cuda()(A)

In [13]:
B

tensor([[[[-0.2982],
          [-0.2431],
          [-0.7081],
          ...,
          [-0.7422],
          [-0.6618],
          [-0.3046]],

         [[ 0.0683],
          [-0.0804],
          [-0.0250],
          ...,
          [ 0.1863],
          [-0.1117],
          [-0.1203]],

         [[-0.3199],
          [-0.1380],
          [-0.1947],
          ...,
          [-0.0441],
          [-0.3234],
          [-0.0039]],

         ...,

         [[-0.0629],
          [ 0.1735],
          [-0.1044],
          ...,
          [-0.3480],
          [ 0.0234],
          [ 0.2862]],

         [[-0.2767],
          [-0.3573],
          [-0.6291],
          ...,
          [-0.4452],
          [-0.4202],
          [-0.4769]],

         [[ 0.2735],
          [ 0.4395],
          [ 0.2507],
          ...,
          [ 0.1173],
          [ 0.3946],
          [ 0.0975]]],


        [[[-0.2974],
          [-0.4128],
          [-0.4716],
          ...,
          [-0.4754],
          [-0.5159],
     